In [37]:
from neo4j import GraphDatabase


In [38]:
import os
from dotenv import  load_dotenv
load_dotenv()
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

In [39]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [40]:
import math

def ingest_movie(tx, title, year, plot, director, cast_list, genre_list):

    # --- Movie ---
    tx.run("""
        MERGE (m:Movie {title: $title, year: $year})
        SET m.plot = $plot
    """, title=title, year=year, plot=plot)

    # --- Director (FIXED) ---
    if isinstance(director, str) and director.strip():
        tx.run("""
            MATCH (m:Movie {title: $title, year: $year})
            MERGE (d:Director {name: $director})
            MERGE (m)-[:DIRECTED_BY]->(d)
        """, director=director.strip(), title=title, year=year)

    # --- Cast ---
    for actor in cast_list:
        if actor.strip():
            tx.run("""
                MATCH (m:Movie {title: $title, year: $year})
                MERGE (a:Actor {name: $actor})
                MERGE (m)-[:FEATURES]->(a)
            """, actor=actor.strip(), title=title, year=year)

    # --- Genre ---
    for genre in genre_list:
        if genre.strip():
            tx.run("""
                MATCH (m:Movie {title: $title, year: $year})
                MERGE (g:Genre {name: $genre})
                MERGE (m)-[:BELONGS_TO]->(g)
            """, genre=genre.strip(), title=title, year=year)


In [41]:
def ingest_dataframe(df):
    with driver.session() as session:
        for _, row in df.iterrows():
            session.execute_write(
                ingest_movie,
                title=row["Title"],
                year=int(row["Release Year"]),
                plot=row["clean_plot"],
                director=row["Director"],
                cast_list=row["cast_list"],
                genre_list=row["genre_list"]
            )




In [42]:
import pandas as pd

df = pd.read_csv("df1_cleaned.csv")
df.head()


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,clean_plot,cast_list,genre_list
0,1901,Kansas Saloon Smashers,American,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...","a bartender is working at a saloon, serving dr...",[],[]
1,1901,Love by the Light of the Moon,American,NaN,NaN,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...","the moon, painted with a smiling face hangs ov...",[],[]
2,1901,The Martyred Presidents,American,NaN,NaN,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...","the film, just over a minute long, is composed...",[],[]
3,1901,"Terrible Teddy, the Grizzly King",American,NaN,NaN,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting just 61 seconds and consisting of two ...,[],[]
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,the earliest known adaptation of the classic f...,[],[]


In [43]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 34884 entries, 0 to 34883
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Release Year      34884 non-null  int64
 1   Title             34884 non-null  str  
 2   Origin/Ethnicity  34884 non-null  str  
 3   Director          33752 non-null  str  
 4   Cast              33429 non-null  str  
 5   Genre             28774 non-null  str  
 6   Wiki Page         34884 non-null  str  
 7   Plot              34884 non-null  str  
 8   clean_plot        34883 non-null  str  
 9   cast_list         34884 non-null  str  
 10  genre_list        34884 non-null  str  
dtypes: int64(1), str(10)
memory usage: 2.9 MB


In [44]:
def normalize_cast(cast):
    """
    Final, strict actor normalization.
    Guarantees: List[str] of clean, atomic actor names.
    """
    if not isinstance(cast, str) or cast.strip() == "":
        return []

    actors = []
    for raw in cast.split(","):
        name = raw.strip()

        # hard reject serialized/list artifacts
        if not name:
            continue
        if "[" in name or "]" in name:
            continue
        if name.startswith("'") or name.endswith("'"):
            name = name.strip("'").strip()

        actors.append(name)

    return actors


In [45]:
def normalize_genre(genre):
    """
    Final, strict genre normalization.
    Guarantees: List[str] of clean, atomic genre names.
    """
    if not isinstance(genre, str) or genre.strip() == "":
        return []

    genres = []
    for raw in genre.split(","):
        name = raw.strip()

        # remove any serialization artifacts
        name = name.strip("[]'\" ").title()

        if name:
            genres.append(name)

    return genres


In [46]:
df["cast_list"] = df["cast_list"].apply(normalize_cast)
df["genre_list"] = df["genre_list"].apply(normalize_genre)

In [47]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 34884 entries, 0 to 34883
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      34884 non-null  int64 
 1   Title             34884 non-null  str   
 2   Origin/Ethnicity  34884 non-null  str   
 3   Director          33752 non-null  str   
 4   Cast              33429 non-null  str   
 5   Genre             28774 non-null  str   
 6   Wiki Page         34884 non-null  str   
 7   Plot              34884 non-null  str   
 8   clean_plot        34883 non-null  str   
 9   cast_list         34884 non-null  object
 10  genre_list        34884 non-null  object
dtypes: int64(1), object(2), str(8)
memory usage: 2.9+ MB


In [48]:

df["cast_list"].apply(type).value_counts()

cast_list
<class 'list'>    34884
Name: count, dtype: int64

In [49]:
df["genre_list"].apply(type).value_counts()


genre_list
<class 'list'>    34884
Name: count, dtype: int64

In [50]:
graph_df = df[
    ["Title", "Release Year", "clean_plot", "cast_list", "genre_list", "Director"]
].copy()


In [51]:
graph_df["Director"] = graph_df["Director"].fillna("").str.strip()

In [52]:
graph_df["genre_list"].head(5)


0    []
1    []
2    []
3    []
4    []
Name: genre_list, dtype: object

In [53]:
any(isinstance(x, str) for x in graph_df["genre_list"])


False

In [55]:
bad_actors = [
    a
    for sublist in df["cast_list"]
    for a in sublist
    if "[" in a or "]" in a or a.startswith("'") or a.endswith("'")
]

assert len(bad_actors) == 0, f"Corrupted actor names found: {bad_actors[:5]}"


In [59]:
import re

def is_valid_genre(g):
    g = g.strip()

    # structural garbage
    if "[" in g or "]" in g:
        return False

    # editorial / descriptive phrases
    banned_phrases = [
        "based on",
        "adapted from",
        "directorial debut",
        "of the same name",
        "citation",
        "novel",
        "play"
    ]

    gl = g.lower()
    if any(p in gl for p in banned_phrases):
        return False

    # likely person-name contamination (many capitalized words)
    tokens = g.split()
    capitalized = sum(t[0].isupper() for t in tokens if t)
    if capitalized >= 4:
        return False

    return True


In [60]:
df["genre_list"] = df["genre_list"].apply(
    lambda genres: [g for g in genres if is_valid_genre(g)]
)


In [ ]:
#ingest_dataframe(graph_df)
#done
